# **Machine Learning que separa o que é verdade e o que é Fake News**



Nesse estudo estarei seguindo as aulas do Canal Codfike onde é ensinado a distinguir as notícias que são reais das que são Fake News.

Link: https://www.youtube.com/watch?v=UMw5-PhRZS0


O Dataset utilizado se encontra no Kaggle: https://www.kaggle.com/c/fake-news/data

In [2]:
#Importando bibliotecas 
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


### **Removendo as Stopwords**

O que são stopwords? As famosas Stopwords (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma Search Engine.

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Stopwords - Inglês**

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Stopwords - Português**

In [6]:
print(stopwords.words('portuguese'))

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

Stopwords = São palavras que não queremos que seja incluidas na nossa String.

Nessa primeira parte da Análise vou trabalhar com Dataset em inglês, dessa forma, utilizarei Stopwords em inglês.






---



### *Importando Dataset do Kaggle* 

In [7]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"keziacampos","key":""}'}

In [8]:
! mkdir ~/.kaggle

In [9]:
! cp kaggle.json ~/.kaggle/

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle competitions download -c fake-news


 85% 8.00M/9.42M [00:00<00:00, 82.6MB/s]
100% 9.42M/9.42M [00:00<00:00, 85.7MB/s]
 97% 36.0M/37.0M [00:00<00:00, 78.6MB/s]
100% 37.0M/37.0M [00:00<00:00, 123MB/s] 
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 38.0MB/s]


In [20]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


### *Fim da Importação do Kaggle*



---



In [21]:
df = pd.read_csv('train.csv', encoding='utf8')

In [22]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
#verificar os nulos
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [25]:
#retirar os nulos
df.dropna(inplace=True)

In [26]:
#verificar os nulos
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [27]:
#juntar os campos
df['all']=df['title']+' '+df['author']+' '+df['text']

In [29]:
#verificar a junção dos campos
pd.options.display.max_colwidth = 150

print(df['all'])

0        House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell Lucus House Dem Aide: We Didn’t Even See Comey’s Letter ...
1        FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart Daniel J. Flynn Ever get the feeling your life circles the roundabout rather than heads in...
2        Why the Truth Might Get You Fired Consortiumnews.com Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analyst...
3        15 Civilians Killed In Single US Airstrike Have Been Identified Jessica Purkiss Videos 15 Civilians Killed In Single US Airstrike Have Been Identi...
4        Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery Howard Portnoy Print \nAn Iranian woman has been sen...
                                                                                 ...                                                                          
20795    Rapper T.I.: Trump a ’Poster Child Fo

In [41]:
#criar função de steming
porter_stemmer = PorterStemmer()

def stemming_tokenizer(df):
  words = re.sub(r"[^A-Za-z ]", " ", df).lower().split()
  words = [porter_stemmer.stem(word) for word in words if not words in stopwords.words('english')] 
  words = ' '.join(words)
  return words  

A função de Steming vai trazer a palavra para a sua raiz.

Exemplo:

correndo -> correr

estudando -> estudar


In [42]:
df['features'] = df['all'].apply(stemming_tokenizer)

In [57]:
print(df['features'])

0        hous dem aid we didn t even see comey s letter until jason chaffetz tweet it darrel lucu hous dem aid we didn t even see comey s letter until jaso...
1        flynn hillari clinton big woman on campu breitbart daniel j flynn ever get the feel your life circl the roundabout rather than head in a straight ...
2        whi the truth might get you fire consortiumnew com whi the truth might get you fire octob the tension between intellig analyst and polit policymak...
3        civilian kill in singl us airstrik have been identifi jessica purkiss video civilian kill in singl us airstrik have been identifi the rate at whic...
4        iranian woman jail for fiction unpublish stori about woman stone to death for adulteri howard portnoy print an iranian woman ha been sentenc to si...
                                                                                 ...                                                                          
20795    rapper t i trump a poster child for w

In [58]:
#separar em features e labels
X = df['features'].values
Y = df['label'].values

In [59]:
Y

array([1, 0, 1, ..., 0, 1, 1])

Vai verificar a contagem de palavras para saber quais palavras repetem mais e quais delas está mais para Fake news e quais não estão

In [60]:
#converter de texto para numero
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [61]:
X = vectorizer.transform(X)

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify= Y, random_state=42 )

In [63]:
#Treinando o modelo
lr= LogisticRegression()
lf.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Acima foi utilizado Logisic Regression (Regressão Logística)**


A regressão logística é um dos algoritmos de aprendizado de máquina mais populares, que vem com a técnica de aprendizado supervisionado. É usado para prever a variável dependente categórica usando um determinado conjunto de variáveis ​​independentes.


A regressão logística prevê a saída de uma variável dependente categórica. Portanto, o resultado deve ser um valor categórico ou discreto. Pode ser Sim ou Não, 0 ou 1, verdadeiro ou Falso, etc., mas em vez de fornecer o valor exato como 0 e 1, fornece os valores probabilísticos que se encontram entre 0 e 1.


A regressão logística é muito semelhante à regressão linear, exceto na forma como são usadas. A regressão linear é usada para resolver problemas de regressão, enquanto a regressão logística é usada para resolver os problemas de classificação.


Na regressão logística, em vez de ajustar uma linha de regressão, ajustamos uma função logística em forma de "S", que prevê dois valores máximos (0 ou 1).
A curva da função logística indica a probabilidade de algo como se as células são cancerosas ou não, um camundongo é obeso ou não com base em seu peso, etc.


A regressão logística é um algoritmo de aprendizado de máquina significativo porque tem a capacidade de fornecer probabilidades e classificar novos dados usando conjuntos de dados contínuos e discretos.

[fonte](https://www.javatpoint.com/logistic-regression-in-machine-learning)

In [64]:
#Previsão
pred= lr.predict(X_test)

acc= accuracy_score(Y_test, pred)

f'Acuracia:{acc * 100:.2f}'

'Acuracia:95.62'

### **O que é Acurácia?**
Indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente;


[fonte: Métricas de Avaliação: acurácia, precisão, recall… quais as diferenças?
](https://https://vitorborbarodrigues.medium.com/m%C3%A9tricas-de-avalia%C3%A7%C3%A3o-acur%C3%A1cia-precis%C3%A3o-recall-quais-as-diferen%C3%A7as-c8f05e0a513c#:~:text=Acur%C3%A1cia%3A%20indica%20uma%20performance%20geral,modelo%20fez%2C%20quantas%20est%C3%A3o%20corretas%3B&text=F1%2DScore%3A%20m%C3%A9dia%20harm%C3%B4nica%20entre%20precis%C3%A3o%20e%20recall.) 

No caso dessa Machine Learning: A acurácia foi de **95.62%**